In [21]:
import atomium
import math
import mdtraj as md
import nglview
import pybel as bel
import os
from tqdm import tqdm
from rmsd import *
import numpy as np
from pmapper.pharmacophore import Pharmacophore as Pharma
from scipy.spatial.transform import Rotation as Rotate
from rdkit import Chem
from rdkit.Chem import AllChem
import deepchem as dc
from sklearn.ensemble import BaggingRegressor
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split
import pandas as pd

from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.metrics import accuracy_score
import time

import numpy as np 
import scipy 
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt 
from mpl_toolkits.mplot3d import Axes3D 
from sklearn.cluster import KMeans
from scipy.spatial.transform import Rotation as R
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import classification_report,f1_score
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import r2_score

import deepchem.utils.rdkit_util as dcu
import atomium
from scipy.spatial.transform import Rotation as R

In [20]:
input_folder = "SI/DockingInput/Ligands_pdb/"
output_folder = "SI/DockingResults/Interface/LigandsOut_0/Ligands_pdb/"
temp_folder = "SI/rotated/"

In [3]:
def trans_rot(P,Q):
    
    p = P.mean(axis = 0)
    q = Q.mean(axis = 0)
    T = q-p   #Translation vector
    P += T
    
    R = kabsch(P,Q)   # Rotation matrix
    R = Rotate.from_matrix(R).as_rotvec()
    return np.concatenate([T, np.ndarray.flatten(R)])
    

In [6]:
def get_XY():
    
    X_fingerprints = []
    X_coulomb = []
    X_desc = []
    Y = []
    X_All = []
    Files = []
    
    for file in tqdm(os.listdir(input_folder)):
        try:
            Initial_molecule = Chem.MolFromPDBFile(input_folder+file)
            Final_molecule = Chem.MolFromPDBFile(output_folder+file)
        
            Chem.rdMolTransforms.CanonicalizeMol(Initial_molecule)   # Aligns principal axes with x, y, z
            Initial_coords = Initial_molecule.GetConformers()[0].GetPositions()

            Final_coords = Final_molecule.GetConformers()[0].GetPositions()

            desc = dc.feat.RDKitDescriptors()
            x_desc = desc.featurize([Initial_molecule]).flatten()
            if np.any(np.isnan(x_desc)): continue
            
            fing = dc.feat.CircularFingerprint()
            x_fingerprint = fing.featurize([Initial_molecule]).flatten()
            if np.any(np.isnan(x_fingerprint)): continue

            coulomb_eig = dc.feat.CoulombMatrixEig(max_atoms=200)
            x_coulomb = coulomb_eig._featurize(Initial_molecule).flatten()
            if np.any(np.isnan(x_coulomb)): continue
        
                
        except:
            continue
            
        X_desc.append(x_desc)
        X_fingerprints.append(x_fingerprint)
        X_coulomb.append(x_coulomb)

        
        Files.append(file)
        X_All.append(np.concatenate([x_fingerprint, x_coulomb, x_desc]))
        Y.append(trans_rot(Initial_coords, Final_coords))
        
        data = {'Pdb File': Files, 'X_All':X_All, 'X_Coulomb': X_coulomb, 'X_Fing': X_fingerprints, 'X_desc': X_desc, 'Y':Y}
        df = pd.DataFrame(data)
        
    return df

In [7]:
df= get_XY()


  1%|          | 108/9121 [00:06<08:43, 17.22it/s]RDKit ERROR: [17:06:45] Explicit valence for atom # 18 C, 5, is greater than permitted
RDKit ERROR: [17:06:45] Explicit valence for atom # 18 C, 5, is greater than permitted
 17%|█▋        | 1515/9121 [01:29<07:05, 17.86it/s]RDKit ERROR: [17:08:08] Explicit valence for atom # 26 O, 3, is greater than permitted
RDKit ERROR: [17:08:08] Explicit valence for atom # 7 O, 3, is greater than permitted
 34%|███▎      | 3061/9121 [03:23<09:25, 10.72it/s]RDKit ERROR: [17:10:02] Explicit valence for atom # 39 O, 3, is greater than permitted


 34%|███▍      | 3087/9121 [03:25<07:37, 13.19it/s]RDKit ERROR: [17:10:04] Explicit valence for atom # 18 C, 5, is greater than permitted
RDKit ERROR: [17:10:04] Explicit valence for atom # 18 C, 5, is greater than permitted
 34%|███▍      | 3094/9121 [03:25<08:18, 12.08it/s]RDKit ERROR: [17:10:04] Explicit valence for atom # 18 C, 5, is greater than permitted
RDKit ERROR: [17:10:04] Explicit valence for atom # 18 C, 5, is greater than permitted
 40%|████      | 3677/9121 [04:09<05:30, 16.49it/s]RDKit ERROR: [17:10:48] Explicit valence for atom # 18 C, 5, is greater than permitted
RDKit ERROR: [17:10:48] Explicit valence for atom # 18 C, 5, is greater than permitted
 42%|████▏     | 3876/9121 [04:25<06:41, 13.08it/s]RDKit ERROR: [17:11:04] Explicit valence for atom # 18 C, 5, is greater than permitted
RDKit ERROR: [17:11:04] Explicit valence for atom # 18 C, 5, is greater than permitted
 43%|████▎     | 3906/9121 [04:27<06:38, 13.10it/s]RDKit ERROR: [17:11:06] Explicit valence for atom

In [2]:
df = pd.read_pickle("all_data.pkl")

In [3]:
X = np.array([df['X_Coulomb'][i] for i in range(len(df))])
Y = np.array([df['Y'][i] for i in range(len(df))])
X_train, X_test, Y_train, Y_test, test_ind, train_ind = train_test_split(X, Y, range(X.shape[0]), test_size = 0.25, random_state=42)

In [4]:
clf = RandomForestRegressor()

In [5]:
clf.fit(X_train, Y_train)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

In [6]:
Y_pred = clf.predict(X_test)

In [7]:
Y_pred

array([[-3.49942683, -2.01710741,  2.53265885, -0.10140057, -0.13395153,
         0.369395  ],
       [-1.065304  , -1.40232215,  1.97115721,  0.31024459, -0.41462933,
         0.44534526],
       [-1.38828456, -2.06265566,  2.16942381,  0.13226326, -0.18918545,
         0.35773842],
       ...,
       [-1.09007619, -1.52241574,  2.33766814, -0.05025439, -0.17780027,
         0.7264052 ],
       [-4.73446951, -2.93424224,  2.72625216,  0.44183469,  0.2917165 ,
        -0.16451708],
       [-2.27339658, -1.79246034,  2.3742386 , -0.15899142, -0.72471861,
         1.55248337]])

In [8]:
r2_score(Y_test, Y_pred)

0.27501380641738865

In [15]:
best  = np.argsort(np.sum((Y_pred - Y_test)**2,axis=1))[0:100]

In [120]:
test_ind[1719]

4510

In [122]:
Y_pred[1719]

array([-4.76218671, -2.9151718 ,  2.70249915, -0.27222731, -0.08167183,
        0.31239018])

In [12]:
best

array([1719, 1712, 1613, 1457, 1256,  918,  303,  725,  941,  126])

In [16]:
best_ind = []
for i in best: best_ind.append(test_ind[i])

In [17]:
best_ind

[4510,
 7092,
 7827,
 5223,
 6703,
 8032,
 6149,
 2526,
 2654,
 1423,
 6795,
 144,
 2705,
 2345,
 3245,
 1873,
 6871,
 7009,
 1349,
 175,
 268,
 5159,
 1052,
 4207,
 2414,
 1225,
 5362,
 1583,
 6388,
 3794,
 1760,
 7803,
 5476,
 166,
 3142,
 5194,
 7121,
 1657,
 4920,
 947,
 7608,
 4513,
 508,
 3757,
 7034,
 1859,
 3622,
 2339,
 6453,
 5639,
 170,
 7447,
 7486,
 8057,
 759,
 3592,
 4231,
 4534,
 7171,
 5544,
 2117,
 861,
 433,
 5689,
 5283,
 7667,
 2763,
 287,
 6381,
 5653,
 3479,
 5441,
 5728,
 7338,
 1416,
 5309,
 1231,
 7261,
 463,
 6382,
 1477,
 5044,
 2771,
 844,
 6773,
 4830,
 4977,
 1255,
 5863,
 5020,
 3881,
 6352,
 7262,
 4967,
 4127,
 5946,
 1964,
 2907,
 2680,
 6076]

In [26]:
# Transforming ligands and storing the final files 
for i in range(1):
    filename = "Lig"+str(best_ind[i])+".pdb"
    print(filename)
    mol = Chem.MolFromPDBFile(input_folder+filename)
    try:
        Chem.rdMolTransforms.CanonicalizeMol(mol) 
    except:
        continue
    dcu.write_molecule(mol, temp_folder+"canon_"+filename)
    
    Y_pred_ind = best[i]
    rot = Y[Y_pred_ind][3:]
    trans = Y[Y_pred_ind][:3]
    r = R.from_rotvec(rot)
    e = r.as_euler('zyx')
    
    canon = atomium.open(temp_folder+"canon_"+filename)
    canon.model.ligand(name='<1>').rotate(e[0], 'z')
    canon.model.ligand(name='<1>').rotate(e[1], 'y')
    canon.model.ligand(name='<1>').rotate(e[2], 'x')
    
    canon.model.ligand(name='<1>').translate(trans)
   
    canon.model.save(temp_folder+"Transformed_"+filename)
    

Lig4510.pdb
Lig7092.pdb
Lig7827.pdb
Lig5223.pdb
Lig6703.pdb
Lig8032.pdb
Lig6149.pdb
Lig2526.pdb
Lig2654.pdb
Lig1423.pdb
Lig6795.pdb
Lig144.pdb
Lig2705.pdb
Lig2345.pdb
Lig3245.pdb
Lig1873.pdb
Lig6871.pdb
Lig7009.pdb
Lig1349.pdb
Lig175.pdb
Lig268.pdb
Lig5159.pdb
Lig1052.pdb
Lig4207.pdb
Lig2414.pdb
Lig1225.pdb
Lig5362.pdb
Lig1583.pdb
Lig6388.pdb
Lig3794.pdb
Lig1760.pdb
Lig7803.pdb
Lig5476.pdb
Lig166.pdb
Lig3142.pdb
Lig5194.pdb
Lig7121.pdb
Lig1657.pdb
Lig4920.pdb
Lig947.pdb
Lig7608.pdb
Lig4513.pdb
Lig508.pdb
Lig3757.pdb
Lig7034.pdb
Lig1859.pdb
Lig3622.pdb
Lig2339.pdb
Lig6453.pdb
Lig5639.pdb
Lig170.pdb
Lig7447.pdb
Lig7486.pdb
Lig8057.pdb
Lig759.pdb
Lig3592.pdb
Lig4231.pdb
Lig4534.pdb
Lig7171.pdb
Lig5544.pdb
Lig2117.pdb
Lig861.pdb
Lig433.pdb
Lig5689.pdb
Lig5283.pdb
Lig7667.pdb
Lig2763.pdb
Lig287.pdb
Lig6381.pdb
Lig5653.pdb
Lig3479.pdb
Lig5441.pdb
Lig5728.pdb
Lig7338.pdb
Lig1416.pdb
Lig5309.pdb
Lig1231.pdb
Lig7261.pdb
Lig463.pdb
Lig6382.pdb
Lig1477.pdb
Lig5044.pdb
Lig2771.pdb
Lig844.pdb
Lig67

RDKit ERROR: [22:09:02] Explicit valence for atom # 18 C, 5, is greater than permitted
